Keras Model with Image Augmentation and Dropout to reduce overfitting

In [ ]:
from keras import layers
from keras import models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
!rm -rf '/content/cats_vs_dogs_small'

In [ ]:
!unzip -qq '/content/drive/MyDrive/Colab_Notebooks/SECONDYEAR_TRIMESTER4_Module03_deep-learning-applications/DAY3_APPLICATION_OF_DEEP_LEARNING/dogs_cats.zip'

In [ ]:
import os, shutil, pathlib

original_dir = pathlib.Path("dogs_cats/train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,dst=dir / fname)
  
make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

In [ ]:
from keras import layers
from keras import models

train_dir="/content/cats_vs_dogs_small/train"
validation_dir="/content/cats_vs_dogs_small/validation"

In [ ]:

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,)

Validation Data Set should not be augmented

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

Resize all images to 150x150

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir,target_size=(150, 150),batch_size=20,class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_dir,target_size=(150, 150),batch_size=20,class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


In [ ]:
from keras import optimizers
model.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(learning_rate=1e-4),metrics=['acc'])

In [ ]:
history = model.fit(
train_generator,
steps_per_epoch=100,
epochs=100,
validation_data=validation_generator,
validation_steps=50)

Epoch 1/100
100/100 [==============================] - 142s 1s/step - loss: 0.6955 - acc: 0.5110 - val_loss: 0.6925 - val_acc: 0.5130
Epoch 2/100
100/100 [==============================] - 141s 1s/step - loss: 0.6877 - acc: 0.5435 - val_loss: 0.6747 - val_acc: 0.5770
Epoch 3/100
100/100 [==============================] - 143s 1s/step - loss: 0.6786 - acc: 0.5660 - val_loss: 0.6947 - val_acc: 0.5470
Epoch 4/100
100/100 [==============================] - 143s 1s/step - loss: 0.6742 - acc: 0.5835 - val_loss: 0.6979 - val_acc: 0.5190
Epoch 5/100
100/100 [==============================] - 135s 1s/step - loss: 0.6583 - acc: 0.6165 - val_loss: 0.6394 - val_acc: 0.6440
Epoch 6/100
100/100 [==============================] - 141s 1s/step - loss: 0.6557 - acc: 0.6180 - val_loss: 0.6263 - val_acc: 0.6680
Epoch 7/100
100/100 [==============================] - 136s 1s/step - loss: 0.6408 - acc: 0.6260 - val_loss: 0.6500 - val_acc: 0.6180
Epoch 8/100
100/100 [==============================] - 142s 1s

In [ ]:
model.save('cats_and_dogs_small_2.h5')

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
!jupyter nbconvert --to html KerasModel_to_reduce_overfitting.ipynb